## Loading import

In [1]:
import random
from loguru import logger

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import plotly.express as px
import open3d as o3d
from open3d.cpu.pybind.visualization import draw_geometries
import matplotlib.colors as mcolors
import copy
import pyransac3d as pyrsc
import laspy

from point_cloud_segmentation.config import RAW_DATA_DIR
from point_cloud_segmentation.visualizing.visualise import *

from open3d.cpu.pybind.visualization import Visualizer, RenderOption



2026-02-05 08:58:23.912 | INFO     | point_cloud_segmentation.config:<module>:11 - PROJ_ROOT path is: D:\wkd\work\point-cloud-segmentation


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Loading data

In [2]:
las = laspy.read(RAW_DATA_DIR / "wkd-house_1_8xds.laz")
# las = laspy.read(RAW_DATA_DIR / "wkd-house_2_8xds.laz")
print(np.unique(las.classification))

points = np.vstack((las.x, las.y, las.z)).transpose()

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

print(pcd)


[0]
PointCloud with 70775329 points.


## Downsampling

In [3]:
voxel_size = 0.05
ds_pcd = pcd.voxel_down_sample(voxel_size=voxel_size)
print(ds_pcd)

PointCloud with 524347 points.


## ASD

In [18]:
ds_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
# ds_pcd_np = np.array(ds_pcd.points)
ds_pcd_np = np.array(pcd.points)
ds_pcd = pcd

print(ds_pcd)
print(ds_pcd_np.shape)


PointCloud with 70775329 points.
(70775329, 3)


In [ ]:
epsilon = voxel_size + voxel_size
min_points = 5
labels = np.array(ds_pcd.cluster_dbscan(eps=epsilon, min_points=min_points, print_progress=True))

print(labels)
print(labels.shape)

inlier_mask = labels >= 0
outlier_mask = labels == -1

inliers = ds_pcd_np[inlier_mask]
outliers = ds_pcd_np[outlier_mask]
print(inliers.shape)
print(outliers.shape)

inliers_pcd = o3d.geometry.PointCloud()
outliers_pcd = o3d.geometry.PointCloud()
inliers_pcd.points = o3d.utility.Vector3dVector(inliers)
outliers_pcd.points = o3d.utility.Vector3dVector(outliers)
print(inliers_pcd)
print(outliers_pcd)

visualize_geometries([inliers_pcd, outliers_pcd])
